In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=10,6

In [ ]:
import statsmodels.api as sm
# i) stationarity
from statsmodels.tsa.stattools import adfuller
# ii) seasonal,trend,residuals
from statsmodels.tsa.seasonal import seasonal_decompose
#iii)acf & pacf
from statsmodels.tsa.stattools import acf,pacf
# Alternative ACF & PACF plots
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
#iv ARIMA model
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima.model import ARIMA
#vi)SARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
#Model tuning
import pmdarima as pm
#test & train set
from pmdarima.model_selection import train_test_split

#warnings
import warnings
warnings.simplefilter('ignore')

In [ ]:
import statsmodels as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.tools import diff 

import pmdarima as pm
from pmdarima.datasets import load_wineind
from pmdarima.arima import AutoARIMA
from pmdarima import auto_arima
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import FourierFeaturizer
from pmdarima.model_selection import train_test_split
from statsmodels.stats.diagnostic import acorr_ljungbox
from pandas.plotting import autocorrelation_plot

import warnings
warnings.simplefilter('ignore')

In [ ]:
def ETL(path):
    df = pd.read_csv(path)
    df.set_index(df.iloc[:,0],inplace=True)
    df.dropna(inplace=True)
    df.parse_dates=True
    df.columns
    df =pd.DataFrame(df['Close'],columns =['Close'])
    df.plot(figsize=(10,6),title='Close Trend')
    
    print(type(df))
    print('Shape',df.shape)
    df.head()
    return df

In [ ]:
df = ETL(r'C:\Users\kelvin\Downloads\archive (31)\HCLTECH.csv')
df.head()

In [ ]:
df.loc['2015-1-1':].plot(figsize=(10,6))

In [ ]:
rollmean = df.rolling(window=12).mean()
rollstd = df.rolling(window=12).std()

plt.figure(figsize=(16,5))
fig = plt.figure(1)

orig = plt.plot(df,color='blue',label='original')
mean = plt.plot(rollmean,color='red',label='mean')
rollstd = plt.plot(rollstd,color='black',label='std')

plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviations')
plt.grid()
plt.show(block = False)

In [ ]:
x=df['Close'].values
result = adfuller(x)
print('ADF Statistic: %f'%result[0])
print('p-value: %f'%result[2])
print('Critical values:')
for key,value in result[4].items():
    print('\t%s:%.3f'%(key,value))
if result[0]<result[4]['5%']:
    print('Reject Ho-Time series is stationary')
else:
    print('Failed to reject-Time series is non-stationary')

In [ ]:
def autocorr_test(series,lags,alpha):
    k=acorr_ljungbox(series,lags=[1],return_df=True)

    if k['lb_pvalue'].values >= alpha:
        print('Fail to Reject the  H0 at',alpha,'Significance level and conclude','The Time Series DOES NOT contain  Autocorrelation')
        print('The p-value of',k['lb_pvalue'].values,'is greater or equal to',alpha)
        print('The statistic value is',k['lb_stat'].values,'in',alpha,'number of lags')
    else:
        print('Reject the H0 at',alpha,'Significance level and conclude','The Time Series contain Autocorrelation')
        print('The p-value of',k['lb_pvalue'].values,'is less than',alpha)
        print('The statistic value is',k['lb_stat'].values,'in',lags,'number of the lags')

autocorr_test(df,12,0.05)

In [ ]:
autocorrelation_plot(df)
plt.show()

In [ ]:
#Differencing

df_log = df['Close'].apply(np.log)
# df_log = np.log(df['Close'])

df_log=diff(df, k_diff=0, k_seasonal_diff=True, seasonal_periods=12)
df_log

In [ ]:
plt.figure(figsize=(16,7))
fig = plt.figure(1)
df_diff = df_log  - df_log.shift(1)
plt.plot(df_diff)

rollmean = df_diff.rolling(12).mean()
rollstd = df_diff.rolling(12).std()



orig = plt.plot(df_diff,color='blue',label='original')
mean = plt.plot(rollmean,color='red',label='Rolling Mean')                
std = plt.plot(rollstd,color='black',label='rolling std')
               
plt.legend(loc='best')
plt.title("Rolling Mean & Standard Deviations") 
plt.plot(block=False)               

In [ ]:
y=df_log.values
result = adfuller(y)
print('ADF Statistic: %f'%result[0])
print('p-value: %f'%result[2])
print('Critical values:')
for key,value in result[4].items():
    print('\t%s:%.3f'%(key,value))
if result[0]<result[4]['5%']:
    print('Reject Ho-Time series is stationary')
else:
    print('Failed to reject-Time series is non-stationary')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
#missing
decomposition = seasonal_decompose(df, model='additive',period=30)


trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(df_log,label = 'Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend,label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='seasonality')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

decomposedLogData=residual
decomposedLogData.dropna(inplace=True)

In [ ]:
residuals =pd.DataFrame(residual)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
residuals.describe()

In [ ]:
from statsmodels.tsa.stattools import acf,pacf
lag_acf = acf(df,nlags=20)
lag_pacf = pacf(df,nlags=20)
print(lag_acf)
print(lag_pacf )

In [ ]:
import statsmodels.api as sm
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df_log.dropna(),lags=30,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df_log.dropna(),lags=30,ax=ax2)

In [ ]:
sm.graphics.tsa.plot_acf(df['Close'].values.squeeze(),lags=30)
sm.graphics.tsa.plot_pacf(df['Close'].values.squeeze(),lags=30)
plt.show()

In [ ]:
stepwise_fit = auto_arima(df['Close'],trace=True,suppress_warnings = True)
stepwise_fit.summary()

In [ ]:
#Alternatively
import pmdarima as pm
def arimamodel(timeseries):
    automodel = pm.auto_arima(timeseries,
                             start_p =2,
                             start_q =2,
                             max_q=10,
                             max_p=10,
                             test ='adf',
                             seasonal=True,
                             trace =True
                             )
    return automodel

In [ ]:
arimamodel(df_log)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model= ARIMA(df,order=(2,0,1))
results_ARIMA = model.fit()

results_ARIMA.summary()

In [ ]:
plt.plot(df_log,color='b')
plt.plot(results_ARIMA.fittedvalues,color='r')
# plt.title('RSS:%.4f'%sum(results_ARIMA.fittedvalues - df_diff)**2)
print('Plotting AR Model')


In [ ]:
y= df['Close'].values
train, test = train_test_split(y, train_size=150)

# Fit your model
model = pm.auto_arima(train, seasonal=True, m=12)

# make your forecasts
forecasts = model.predict(test.shape[0])  # predict N steps into the future

# Visualize the forecasts (blue=train, green=forecasts)
plt.figure(figsize=(16,8))

x = np.arange(y.shape[0])
plt.plot(x[:150], train, c='blue')
plt.plot(x[150:], forecasts, c='green')

plt.show()
print(forecasts)

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(y,  c='green')
plt.plot(x[:150], train, c='red')
plt.plot(x[150:], forecasts, c='blue')

In [ ]:
plt.figure(figsize=(18,6))

plt.plot(y,  c='blue')
plt.plot(x[150:], forecasts, c='red')

In [ ]:
a =pd.DataFrame(forecasts,columns=['Forecast'])
b=pd.DataFrame(y[150:],columns=['Actual'])
ab =pd.concat([a,b],axis=1)
ab['Forecast'].plot(figsize=(16,8),color='red',label='Forecast')
ab['Actual'].plot(figsize=(16,8),color='blue',label='Actual')
plt.legend(loc='best')

In [ ]:
df_log.plot()
plt.plot(np.zeros(len(df)))

In [ ]:
# X = df_log
# size = int(len(X)* 0.66)
# train,test = X[0:size],X[size:len(X)]
# predictions = list()
# history = [x for x in train]

# for t in range(len(test)):
#                model = ARIMA(history,order=(5,1,0)).fit()

In [ ]:
size

In [ ]:
 df_log.dropna(inplace=True)

In [ ]:
df_log